# Velocity dispersion influence of ULDM soliton

In this notebook we try to see the influence in the velocity dispersion when a core given by the soliton of Ultra Light Dark Matter is added in a Mass sheet degenerate way.
This notebook is inspired from [this notebook](https://github.com/TDCOSMO/hierarchy_analysis_2020_public/blob/master/MST_impact/MST_pl_cored.ipynb) from S. Birrer.

Use [this fork](https://github.com/lucateo/lenstronomy/tree/uldm-teodori-inverse-thetac) of lenstronomy to make this run, in order not to have problems.

## Import standard Python libraries and lenstronomy utils.

In [ ]:
import numpy as np
import os
import time
import copy
import astropy.io.fits as pyfits
from scipy.optimize import fsolve

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.Cosmo.lens_cosmo import LensCosmo
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.PointSource.point_source import PointSource
from lenstronomy.ImSim.image_model import ImageModel
import lenstronomy.Util.param_util as param_util
import lenstronomy.Util.simulation_util as sim_util
import lenstronomy.Util.image_util as image_util
from lenstronomy.Util import kernel_util
from lenstronomy.Data.imaging_data import ImageData
from lenstronomy.Data.psf import PSF

import lenstronomy.Util.constants as const
from lenstronomy.LensModel.Profiles.pemd import PEMD
from lenstronomy.LensModel.Profiles.uldm_pl import Uldm_PL

## Setting cosmology and data specifics

In [ ]:
np.random.seed(42)
# define lens configuration and cosmology
z_lens = 0.5
z_source = 1.5
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=74.3, Om0=0.31, Ob0=0.)
cosmo_cmb = FlatLambdaCDM(H0=67.4, Om0=0.31, Ob0=0.)
lens_cosmo = LensCosmo(z_lens = z_lens, z_source = z_source, cosmo = cosmo)
lens_cosmo_cmb = LensCosmo(z_lens = z_lens, z_source = z_source, cosmo = cosmo_cmb)

# data specifics
sigma_bkg = .05  #  background noise per pixel (Gaussian)
exp_time = 100.  #  exposure time (arbitrary units, flux per pixel is in units # photons/exp_time unit)
numPix = 100  #  cutout pixel size
deltaPix = 0.05  #  pixel size in arcsec (area per pixel = deltaPix**2)
fwhm = 0.1  # full width half max of PSF (only valid when psf_type='gaussian')
psf_type = 'GAUSSIAN'  # 'GAUSSIAN', 'PIXEL', 'NONE'; point spread function
kernel_size = 91


# generate the psf variables
kwargs_psf = {'psf_type': psf_type, 'pixel_size': deltaPix, 'fwhm': fwhm}
#kwargs_psf = sim_util.psf_configure_simple(psf_type=psf_type, fwhm=fwhm, kernelsize=kernel_size, deltaPix=deltaPix, kernel=kernel)
psf_class = PSF(**kwargs_psf)

## Defining lens model default values

In [ ]:
# lensing quantities
gamma1, gamma2 = param_util.shear_polar2cartesian(phi=-0.5, gamma=0.09) # shear quantities
kwargs_shear = {'gamma1': gamma1, 'gamma2': gamma2}  # shear values

theta_E = 1.66
kappa_0 = 0.1
slope = 3.78
inverse_theta_c = 0.1
gamma = 1.98

kwargs_spemd = {'theta_E': theta_E, 'gamma': gamma, 'center_x': 0.0, 'center_y': 0.0, 'e1': 0.05, 'e2': 0.05}  # parameters of the deflector lens model
kwargs_spemd_MST = {'theta_E': theta_E*(1 - kappa_0), 'gamma': 1.98, 'center_x': 0.0, 'center_y': 0.0, 'e1': 0.05, 'e2': 0.05}  # parameters of the deflector lens model
kwargs_uldm = {'kappa_0': kappa_0, 'inverse_theta_c': inverse_theta_c, 'slope': slope, 'center_x': 0.0, 'center_y': 0.0}  # parameters of the deflector lens model

lens_model_list = ['PEMD', 'SHEAR']
lens_model_list_uldm = ['PEMD', 'SHEAR', 'ULDM']

kwargs_lens = [kwargs_spemd, kwargs_shear]
kwargs_lens_uldm = [kwargs_spemd_MST, kwargs_shear, kwargs_uldm]

lens_model_class = LensModel(lens_model_list=lens_model_list, z_lens=z_lens, z_source=z_source, cosmo=cosmo)
lens_model_class_uldm = LensModel(lens_model_list=lens_model_list_uldm, z_lens=z_lens, z_source=z_source, cosmo=cosmo_cmb)

# choice of source type
source_type = 'SERSIC'  # 'SERSIC' or 'SHAPELETS'
source_x = 0.
source_y = 0.1
# Sersic parameters in the initial simulation for the source
phi_G, q = 0.5, 0.8
e1, e2 = param_util.phi_q2_ellipticity(phi_G, q)
kwargs_sersic_source = {'amp': 2000, 'R_sersic': 0.1, 'n_sersic': 1, 'e1': e1, 'e2': e2, 'center_x': source_x, 'center_y': source_y}
#kwargs_else = {'sourcePos_x': source_x, 'sourcePos_y': source_y, 'quasar_amp': 400., 'gamma1_foreground': 0.0, 'gamma2_foreground':-0.0}
source_model_list = ['SERSIC_ELLIPSE']
kwargs_source = [kwargs_sersic_source]
source_model_class = LightModel(light_model_list=source_model_list)

# lens light model
phi_G, q = 0.9, 0.9
e1, e2 = param_util.phi_q2_ellipticity(phi_G, q)
kwargs_sersic_lens = {'amp': 4000, 'R_sersic': 0.4, 'n_sersic': 2., 'e1': e1, 'e2': e2, 'center_x': 0.0, 'center_y': 0}
lens_light_model_list = ['SERSIC_ELLIPSE']
kwargs_lens_light = [kwargs_sersic_lens]
lens_light_model_class = LightModel(light_model_list=lens_light_model_list)

point_source_list = ['LENSED_POSITION']
point_source_class = PointSource(point_source_type_list=point_source_list, fixed_magnification_list=[False])

kwargs_numerics = {'supersampling_factor': 1}

## Setting the kinematics

In [ ]:
# observational conditions of the spectroscopic campagne
R_slit = 1. # slit length in arcsec
dR_slit = 1.  # slit width in arcsec
psf_fwhm = 0.7

kwargs_aperture = {'aperture_type': 'slit', 'length': R_slit, 'width': dR_slit, 'center_ra': 0.05, 'center_dec': 0, 'angle': 0}
anisotropy_model = 'OM' # Osipkov-Merritt
aperture_type = 'slit'

kwargs_numerics_galkin = {'interpol_grid_num': 1000,  # numerical interpolation, should converge -> infinity
                          'log_integration': True,  # log or linear interpolation of surface brightness and mass models
                           'max_integrate': 100, 'min_integrate': 0.001}  # lower/upper bound of numerical integrals

r_ani = 1.
r_eff = 0.2
kwargs_anisotropy = {'r_ani': r_ani}
kwargs_seeing = {'psf_type': 'GAUSSIAN', 'fwhm': psf_fwhm}
kwargs_model = {'lens_model_list': lens_model_list,
                 'lens_light_model_list': lens_light_model_list,
                 'source_light_model_list': source_model_list,
                'point_source_model_list': point_source_list
                 }
kwargs_model_uldm = {'lens_model_list': lens_model_list_uldm,
                 'lens_light_model_list': lens_light_model_list,
                 'source_light_model_list': source_model_list,
                'point_source_model_list': point_source_list
                 }

from lenstronomy.Analysis.kinematics_api import KinematicsAPI
kin_api = KinematicsAPI(z_lens, z_source, kwargs_model, cosmo=cosmo,
                        # Put the appropriate bools here if you change the number of lenses!!
                        lens_model_kinematics_bool=[True, False], light_model_kinematics_bool=[True],
                        kwargs_aperture=kwargs_aperture, kwargs_seeing=kwargs_seeing,
                        anisotropy_model=anisotropy_model, kwargs_numerics_galkin=kwargs_numerics_galkin,
                        sampling_number=10000,  # numerical ray-shooting, should converge -> infinity
                        Hernquist_approx=True)

kin_api_uldm = KinematicsAPI(z_lens, z_source, kwargs_model_uldm, cosmo=cosmo_cmb,
                        # Put the appropriate bools here if you change the number of lenses!!
                        lens_model_kinematics_bool=[True, False, True], light_model_kinematics_bool=[True],
                        kwargs_aperture=kwargs_aperture, kwargs_seeing=kwargs_seeing,
                        anisotropy_model=anisotropy_model, kwargs_numerics_galkin=kwargs_numerics_galkin,
                        sampling_number=10000,  # numerical ray-shooting, should converge -> infinity
                        Hernquist_approx=True)

Some function needed to implement the relations between soliton and power law background

In [ ]:
def find_tilde_theta(x, *arguments):

    tilde_theta = x
    kappa_tilde, inverse_theta_c, theta_E = arguments
    kappa_E = Uldm_PL()._kappa_E(theta_E, tilde_theta, kappa_tilde, inverse_theta_c)
    eq1 = (1 - kappa_E)*theta_E - tilde_theta
    return eq1

def tilde_theta_finder(kappa_tilde, inverse_theta_c, theta_E):

    arguments = (kappa_tilde, inverse_theta_c, theta_E)
    tilde_theta = fsolve(find_tilde_theta, 1.6, args=arguments)
    return tilde_theta[0]

We define here the function which will make the velocity dispersion arrays we will plot.

In [ ]:
def velocity_dependence(kwargs_lens, kappa_list, inverse_theta_c):

    kwargs_lens_result = copy.deepcopy(kwargs_lens)

    vel_disp_list = []
    kappa_E_list = []

    kwargs_lens_kin = copy.deepcopy(kwargs_lens)
    kwargs_lens_kin[2]['kappa_0'] = 0
    kwargs_lens_kin[2]['inverse_theta_c'] = inverse_theta_c
    kwargs_lens_kin[0]['theta_E'] = theta_E 
    vel_disp_0 = kin_api.velocity_dispersion(kwargs_lens_kin, kwargs_lens_light, kwargs_anisotropy, r_eff=r_eff, theta_E=None,)

    for kappa_tilde in kappa_list:
        kappa_ext = 0
        kwargs_lens_kin = copy.deepcopy(kwargs_lens)
        tilde_theta = tilde_theta_finder(kappa_tilde, inverse_theta_c, theta_E)
        if tilde_theta < 0:
            tilde_theta = 0
        kappa_E = 1 - tilde_theta/theta_E
        kappa_0 = Uldm_PL()._kappa_0_real(tilde_theta, kappa_tilde, inverse_theta_c)
        kwargs_lens_kin[2]['kappa_0'] = kappa_0
        kwargs_lens_kin[2]['inverse_theta_c'] = inverse_theta_c
        kwargs_lens_kin[0]['theta_E'] = tilde_theta

        vel_disp = kin_api_uldm.velocity_dispersion(kwargs_lens_kin, kwargs_lens_light, kwargs_anisotropy, r_eff=r_eff, theta_E=None, kappa_ext=kappa_ext)
        vel_disp_list.append(vel_disp)
        kappa_E_list.append(kappa_E)
    return np.array(vel_disp_list), vel_disp_0, np.array(kappa_E_list)

# Plot with theta_c, kappa_E
num_points_kappa = 30
kappa_list = np.linspace(0.001, 0.15, num_points_kappa)
inverse_theta_c_list = [0.2, 0.1, 0.05]
vel_disp_list_r = []
vel_disp_0_r = []
kappa_E_list_r = []
for inverse_theta_core in inverse_theta_c_list:
    vel_disp_list, vel_disp_0, kappa_E_list = velocity_dependence(kwargs_lens_uldm, kappa_list, inverse_theta_core)
    vel_disp_list_r.append(vel_disp_list)
    vel_disp_0_r.append(vel_disp_0)
    kappa_E_list_r.append(kappa_E_list)
    print(inverse_theta_core)

## Plotting the result

In [ ]:
f, axes = plt.subplots(2, 1, gridspec_kw={'height_ratios': [4, 2]}, sharex=True)

for i in range(len(inverse_theta_c_list)):
    vel_disp_list = vel_disp_list_r[i]
    vel_disp_0 = vel_disp_0_r[i]
    kappa_E_list = kappa_E_list_r[i]

    axes[0].plot(kappa_E_list, vel_disp_list, label=r'$\sigma^{\rm P}$ for $\theta_{\rm c} =$'+str(1/inverse_theta_c_list[i])+repr('') )
axes[0].plot(kappa_E_list, vel_disp_0*np.ones(num_points_kappa), 'k--', label=r"$\sigma^{\rm P} = \sigma^{\rm P}(\kappa_\lambda = 0)$")
axes[0].set_ylim([235, 275])
axes[0].set_ylabel(r'$\sigma^{\rm P}$ [km/s]', fontsize = 14)
axes[0].legend(fontsize=14)
axes[0].grid()

for i in range(len(inverse_theta_c_list)):
    vel_disp_list = vel_disp_list_r[i]
    vel_disp_0 = vel_disp_0_r[i]
    kappa_E_list = kappa_E_list_r[i]

    axes[1].plot(kappa_E_list, (vel_disp_list - vel_disp_0) / vel_disp_0 , label=r'$\sigma^{\rm P}(\kappa_{\lambda_{\rm c}})$ for $\theta_{\rm c} =$'+str(1/inverse_theta_c_list[i]))
axes[1].set_ylabel(r'$\Delta \sigma^{\rm P} / \sigma^{\rm P} $', fontsize = 14)
axes[1].plot([0, 2], [0, 0], ':k')
axes[1].set_ylim([-0.10, 0.05])
plt.xlim([0.01, 0.2])
plt.xlabel(r'$\kappa_{\lambda} (\theta_{\rm E}) $', fontsize=14)
plt.tight_layout()
plt.grid()
